In [3]:
import pandas as pd
import math


In [4]:
!kaggle datasets download -d gauravmalik26/food-delivery-dataset

Dataset URL: https://www.kaggle.com/datasets/gauravmalik26/food-delivery-dataset
License(s): other
 51%|███████████████████▍                  | 1.00M/1.95M [00:00<00:00, 1.78MB/s]
100%|██████████████████████████████████████| 1.95M/1.95M [00:00<00:00, 3.03MB/s]


In [5]:
!unzip food-delivery-dataset.zip

Archive:  food-delivery-dataset.zip
  inflating: Sample_Submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [236]:
data = pd.read_csv('train.csv').dropna().drop_duplicates()

print(data.columns)

data_of_interest = data[["Restaurant_longitude", "Restaurant_latitude","Delivery_location_longitude", "Delivery_location_latitude","Road_traffic_density", "Vehicle_condition","Type_of_vehicle","Weatherconditions", "Time_taken(min)"]]

data_of_interest

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken(min)'],
      dtype='object')


,Restaurant_longitude,Restaurant_latitude,Delivery_location_longitude,Delivery_location_latitude,Road_traffic_density,Vehicle_condition,Type_of_vehicle,Weatherconditions,Time_taken(min)
0,75.892471,22.745049,75.912471,22.765049,High,2,motorcycle,conditions Sunny,(min) 24
1,77.683237,12.913041,77.813237,13.043041,Jam,2,scooter,conditions Stormy,(min) 33
2,77.678400,12.914264,77.688400,12.924264,Low,0,motorcycle,conditions Sandstorms,(min) 26
3,76.976494,11.003669,77.026494,11.053669,Medium,0,motorcycle,conditions Sunny,(min) 21
4,80.249982,12.972793,80.289982,13.012793,High,1,scooter,conditions Cloudy,(min) 30
...,...,...,...,...,...,...,...,...,...
45588,75.794257,26.902328,75.804257,26.912328,High,1,motorcycle,conditions Windy,(min) 32
45589,0.000000,0.000000,0.070000,0.070000,Jam,0,motorcycle,conditions Windy,(min) 36
45590,80.242439,13.022394,80.272439,13.052394,Low,1,scooter,conditions Cloudy,(min) 16
45591,76.986241,11.001753,77.026241,11.041753,High,0,motorcycle,conditions Cloudy,(min) 26


In [220]:
def calc_dist_with_haversine(coord1: object, coord2: object):

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    return round(km, 3)


restaurant_location = data[['Restaurant_longitude', 'Restaurant_latitude']].loc[1]
# restaurant_location = data[['Restaurant_latitude', 'Restaurant_longitude']].loc[1]
# delivery_location = data[['Delivery_location_latitude','Delivery_location_longitude']].loc[1]
delivery_location = data[['Delivery_location_longitude','Delivery_location_latitude']].loc[1]

calc_dist_with_haversine(restaurant_location, delivery_location)


20.184

In [221]:
def calc_restaurant_to_delivery_distance(row):
    restaurant_location = row[['Restaurant_longitude', 'Restaurant_latitude']]
    delivery_location = row[['Delivery_location_longitude','Delivery_location_latitude']]

    return calc_dist_with_haversine(restaurant_location,delivery_location)


data_of_interest["distance_km"] = data.apply(calc_restaurant_to_delivery_distance, axis=1)
data_of_interest["Time_taken(min)"] = pd.to_numeric(data_of_interest["Time_taken(min)"].str.replace('(min)',''), errors="coerce")

scooter = data_of_interest.loc[data_of_interest["Type_of_vehicle"].str.strip() == "scooter"]


# t = scooter.loc[scooter["Type_of_vehicle"].str.strip() == '']
pd.set_option('display.max_rows', scooter.shape[0]+1)

# pd.to_numeric(scooter["Time_taken(min)"], errors="coerce").isna()
scooter["distance_km"]

/tmp/ipykernel_78603/2399191262.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_of_interest["distance_km"] = data.apply(calc_restaurant_to_delivery_distance, axis=1)
/tmp/ipykernel_78603/2399191262.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_of_interest["Time_taken(min)"] = pd.to_numeric(data_of_interest["Time_taken(min)"].str.replace('(min)',''), errors="coerce")


1           20.184
4            6.210
6           16.600
11          13.788
13          19.397
15           6.218
22          20.181
26          13.682
32           6.176
33          17.298
34          19.915
36           6.128
37           1.520
38           3.073
39           3.105
42          12.237
44          16.826
45           3.027
47           9.315
49           1.520
51          17.079
57          12.580
58           1.490
60          11.917
61           4.664
62          10.641
64           6.129
67           3.145
68           6.290
69          13.972
70           6.081
71           6.081
74           6.211
78          13.973
80          17.078
82          17.073
87           6.176
92        6048.631
93           3.025
99           6.218
101          4.610
106         16.831
107         20.208
108          3.041
111          2.985
114          7.763
121         16.635
122          6.232
124          7.461
125         20.254
126         16.601
129          6.119
130         

In [228]:
scooter["distance_km"][1]

20.184

In [231]:
def average_speed_by_distance_and_time(total_kilometers: float, total_time: float):
    return total_kilometers / total_time

def calc_restaurant_to_delivery_distance(row):
    return calc_dist_with_haversine(row[['Restaurant_longitude', 'Restaurant_latitude']], row[['Delivery_location_latitude','Delivery_location_longitude']])

#mangler fra restaurant og til destination?
#her er det tiden vi skal have fat i og ikke gennemsnit hastighed.

# def estimate_time_by_estimators(total_distance: calc_dist_with_haversine, weather: str, traffic: str, type_of_vechicle: str, vehicle_condition: str):
#     scooter = data[data['Type_of_vehicle'].str.strip() == 'scooter']
#     return average_speed_by_distance_and_time(total_distance, scooter.sum)

def estimate_time_by_estimators(total_distance: float):
    scooter = data_of_interest.loc[data_of_interest["Type_of_vehicle"].str.strip() == "scooter"]
    print(scooter["distance_km"].sum())

    return average_speed_by_distance_and_time(scooter['distance_km'].sum(), scooter['Time_taken(min)'].sum())


# def estimate_time_by_estimators(total_distance: float):
#     scooter = data_of_interest[data_of_interest['Type_of_vehicle'].str.strip() == 'scooter']
#     return scooter['Time_taken(min)'].sum()

distance = scooter["distance_km"][1]


scooter = data_of_interest[data_of_interest['Type_of_vehicle'].str.strip() == 'scooter']

t = scooter['Time_taken(min)'].sum(min_count=1)

tq = estimate_time_by_estimators(distance)

tq

1681796.225


4.497166134535576

In [233]:
time = distance / tq

time

4.488159742420637

In [235]:
distance = tq * time

distance

20.184